[![View notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bartman_et_al_2021/demo.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Bartman_et_al_2021/demo.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bartman_et_al_2021/demo.ipynb)

In [8]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [10]:
from PySDM_examples.Arabas_et_al_2015 import Settings, SpinUp
from PySDM_examples.Szumowski_et_al_1998 import Simulation, Storage
from PySDM_examples.Szumowski_et_al_1998.plots import _TimeseriesPlot, _ImagePlot
from PySDM_examples.utils import ProgBarController
from PySDM_examples.utils.widgets import HTML, display
import PySDM.products as PySDM_products
from PySDM.exporters import NetCDFExporter
from open_atmos_jupyter_utils import TemporaryFile
from PySDM.physics import si
import os
import numpy as np
from scipy.io import netcdf_file
from matplotlib import pyplot, rcParams
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import GridSpec

In [ ]:
settings = Settings()

cloud_range = (settings.aerosol_radius_threshold, settings.drizzle_radius_threshold)
products = [
    PySDM_products.ParticleSpecificConcentration(name='n_a_mg', unit='mg^-1', radius_range=(0, cloud_range[0])),
    PySDM_products.EffectiveRadius(name='r_eff', unit='um', radius_range=(cloud_range[0], np.inf)),
    PySDM_products.ParticleConcentration(name='n_d_cm3', unit='cm^-3', radius_range=(cloud_range[1], np.inf)),
    PySDM_products.WaterMixingRatio(name='qt', unit='g/kg', radius_range=cloud_range),
    PySDM_products.CondensationTimestepMin(name='dt_cond_min'),
    PySDM_products.CollisionTimestepMin(name='dt_coal_min')
]

settings.n_sd_per_gridbox = 128 if 'CI' not in os.environ else 32
settings.grid = (32, 32)
settings.dt = 32 * si.second
settings.simulation_time = .175 * settings.spin_up_time
settings.output_interval = 1 * si.minute
settings.condensation_rtol_x = 1e-6
settings.condensation_rtol_thd = 5e-7

settings.condensation_dt_cond_range = (.25*si.s, settings.dt)
settings.coalescence_dt_coal_range = settings.condensation_dt_cond_range

settings.mode_1.norm_factor *= 3
settings.mode_2.norm_factor *= 3
settings.spectrum_per_mass_of_dry_air.norm_factor *= 3

storage = Storage()
simulation = Simulation(settings, storage, SpinUp=SpinUp)
simulation.reinit(products)

In [ ]:
simulation.run(ProgBarController())

In [ ]:
temp_file = TemporaryFile('.nc')
exporter = NetCDFExporter(storage, settings, simulation, temp_file.absolute_path)
exporter.run(ProgBarController())

In [ ]:
default_figsize = rcParams["figure.figsize"]
figsize = (1.75 * default_figsize[0], 3.1* default_figsize[1])
gs = GridSpec(nrows=27, ncols=18)

fig = pyplot.figure(figsize=figsize)
na = gs.nrows-19
nb = gs.nrows-11
nc = gs.nrows-3
axs = (
    fig.add_subplot(gs[:na, 1:gs.ncols//2-1]),
    fig.add_subplot(gs[:na, gs.ncols//2+1:2*gs.ncols//2-1]),

    fig.add_subplot(gs[na:nb, 1:gs.ncols//2-1]),
    fig.add_subplot(gs[na:nb, gs.ncols//2+1:2*gs.ncols//2-1]),
    
    fig.add_subplot(gs[nb:nc, 1:gs.ncols//2-1]),
    fig.add_subplot(gs[nb:nc, gs.ncols//2+1:2*gs.ncols//2-1]),
    
    fig.add_subplot(gs[gs.nrows-3:-1,2:-1])
)
gs.tight_layout(fig)
ncdf = netcdf_file(temp_file.absolute_path, mode='r', mmap=False)

# TODO #419: we should not use products here - all info should be obtained from netCDF
plots = []
for var, cmap in {
    'n_a_mg': 'summer',
    'n_d_cm3': 'bone_r',
    'r_eff': 'ocean_r',
    'qt': 'Blues',
    'dt_cond_min': 'tab20c',
    'dt_coal_min': 'tab20c'
}.items():
    plots.append(
        _ImagePlot(fig, axs[len(plots)], grid=ncdf.grid, size=ncdf.size, product=simulation.products[var], cmap=cmap)
    )
plots.append(_TimeseriesPlot(fig, axs[-1], ncdf.variables['T'][:], show=False))

plots[-1].ax.axvline(ncdf.n_spin_up * ncdf.dt)
plots[-1].ax.set_ylim(0, .001)

interval = 100 #ms
frame_list = np.arange(ncdf.variables['T'].shape[0], dtype=int)

def update(frame_num):
    step = frame_num*ncdf.steps_per_output_interval
    
    for i, product in enumerate(('n_a_mg', 'n_d_cm3', 'r_eff', 'qt', 'dt_cond_min', 'dt_coal_min')):
        plots[i].update(ncdf.variables[product][frame_num], step=step)
    
    precip = np.full_like(ncdf.variables['surf_precip'][:], np.nan)
    precip[0:frame_num+1] = ncdf.variables['surf_precip'][0:frame_num+1]
    plots[-1].update(precip)
    
    return (
        plots[0].im, plots[1].im, plots[2].im, 
        plots[3].im, plots[4].im, plots[5].im, 
        plots[-1].timeseries
    )
pyplot.close(fig)

In [ ]:
animation = FuncAnimation(fig, update, frames=frame_list, interval=interval, blit=False)

if 'CI' not in os.environ:
    display(HTML(animation.to_html5_video()))
    file = TemporaryFile('.gif')
    animation.save(file.absolute_path)
    display(file.make_link_widget())

In [ ]:
# save last frame in vector format
svg_file = TemporaryFile('.svg')
fig.savefig(svg_file.absolute_path)
display(svg_file.make_link_widget())